In [1]:
# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

# Magic
%matplotlib inline

# Libraries in use
import sys
import math
import alphalens
import numpy as np
import pandas as pd

# My libraries
sys.path.append('../../infra/')
from FactorGenerator  import FactorGenerator
from FactorEvaluation import FactorEvaluation
from AssetsData import AssetsData

In [2]:
'''
Factor function gets as argument asset specific windowed DF.
DF consits of #window days with all the fields, 
where the last day is the target day, to which the calculated factor will be assigned.
For days that are closer to the beginning of the dataset then #window days, input window DF is pruned.
'''
from sklearn import linear_model
info = AssetsData()

def lassolars(df):
    days_num, _ = df.shape
    # If we are closer to the beginning of the dataframe than 60 days, then let the factor value be NaN 
    if (days_num < 10):
        return np.nan
    # Otherwise return the difference between returns of the first day of the window DF and the last but one
    x, y = info.get_x_y(df.iloc[0:-2])
    reg = linear_model.LassoLars(alpha=1e-10)
    reg.fit(x,y)
    factors = [x for x in list(df.iloc[-1].keys()) if x != 'y']
    x_ = [np.array(df.iloc[-1][factors])]
    return reg.predict(x_)[0]

In [ ]:
df = pd.read_csv("../../data/processed_init_data/processed_rolling_data.csv").set_index(['TIMESTAMP', 'ID'])
del df['Unnamed: 0']

In [ ]:
gen = FactorGenerator()
lasso_predictions  = gen.create_factor(df, lassolars, 'lassolars', window = 60, save = False)